In [1]:
import func.Yaseen_model

In [1]:
from func.Yaseen_model_1 import CombinedModel
import torch
device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:

model = CombinedModel()
model.load_state_dict(torch.load('model/data_3/train4_pytorch-3-data0.8-0-5-7.pth'))
model.to(device)

/tmp/ipykernel_2783479/2880902566.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model/data_3/train4_pytorch-3-data0.8-0-5-7.pth'))


CombinedModel(
  (cnn_model_1): CNNModel_1(
    (conv1): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
    )
    (conv2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (conv3): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (cnn_model_2): CNNModel_2(
    (conv5): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (bn5): BatchNorm2d(64, eps=1e-05

In [3]:
# 加载数据集
import numpy as np
test_data = np.load('/home/qziedu/pcg/train_data/data_3/test/test_data.npz')

In [4]:
test_x = test_data['features']
test_y = test_data['labels']
test_x = torch.tensor(test_x,dtype=torch.float32)
test_y = torch.tensor(test_y,dtype=torch.float32)
print(test_x.shape)
print(test_y.shape)

torch.Size([696, 1, 256, 256])
torch.Size([696])


In [5]:
import torch
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
batch_size = 32
# 模型推理代码
def evaluate_model(model):
    model.eval()  # 设置模型为评估模式
    predictions = []
    true_labels = []
    correct_val = 0
    total_val = 0
    with torch.no_grad():  # 关闭梯度计算
        for i in range(0, len(test_x), batch_size):
            x_mfcc_batch = test_x[i:i+batch_size].to(device)
            y_batch = test_y[i:i+batch_size].to(device)

            # 获取预测结果
            outputs = model(x_mfcc_batch)
            _, predicted = torch.max(outputs, 1)  # 获取每行的最大值作为预测类别
            correct_val += (predicted==y_batch).sum().item()
            #print( correct_val)
            total_val += y_batch.size(0)
            
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(y_batch.cpu().numpy())
        val_accuracy = correct_val / total_val
    return np.array(predictions), np.array(true_labels),val_accuracy

# 调用evaluate_model函数获取预测值和真实标签
predictions, true_labels,val_accuracy = evaluate_model(model)
print(val_accuracy)
print(type(predictions))



0.992816091954023
<class 'numpy.ndarray'>
